In [ ]:
# Imports and definitions

import keras
import numpy as np
from datetime import datetime

from os import listdir, makedirs
from os.path import exists, isfile, join

model_name = "proof-of-concept"


In [ ]:
# Create model

lines_shape = (4,)

lines_input = keras.layers.Input(shape=lines_shape, name="regression: proof of concept")
concatenated = keras.layers.Concatenate()([lines_input, lines_input])

output = keras.layers.Dense(1, activation = "linear", name = "output")(concatenated)

model = keras.models.Model(inputs=[lines_input, lines_input], outputs=output)
model.compile(optimizer="adam", loss="mean_squared_error", metrics=["mae"])
model.summary()


In [ ]:
# Show weights and biases of the model

model_parameters = model.get_layer('output').get_weights()
weights, biases = model_parameters
print("biases:\n", biases)
print("weights:\n", weights)


In [ ]:
# Save model

if not exists(f"models/{model_name}"):
    makedirs(f"models/{model_name}")
model.save(f"models/{model_name}/{datetime.today().strftime('%Y-%m-%d_%H-%M-%S')}.keras")


In [ ]:
# Train model

evaluations = np.load("evaluations.npy")
player1_lines = np.load("player1_lines.npy")
player2_lines = np.load("player2_lines.npy")

while True:
    model_version = sorted([f for f in listdir(f"models/{model_name}") if isfile(join(f"models/{model_name}", f))])[-1]

    model = keras.models.load_model(f"models/{model_name}/{model_version}")
    model.compile(optimizer="adam", loss="mean_squared_error", metrics=["mae"])
    model.fit([player1_lines, player2_lines], evaluations, epochs = 10, batch_size = 128, validation_split = 0.2)

    model.save(f"models/{model_name}/{datetime.today().strftime('%Y-%m-%d_%H-%M-%S')}.keras")
